import required packages and setup environment

In [1]:
import arcpy, os, re
from arcpy.sa import *
import pandas as pd

wd = os.getcwd()
arcpy.env.workspace = os.path.join(wd, "DataExploration.gdb")
#gdbLocation = os.path.join(wd, "geodatabases")
scratch = os.path.join(wd, "scratch")
outputs = os.path.join(wd, "outputs")

arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 BC Environment Albers")
arcpy.env.overwriteOutput = True
arcpy.env.snapRaster = r"H:\Muise\forestMask\ForMask_UTM-7S-11S_1984.tif"
arcpy.env.cellSize = 30
arcpy.CheckOutExtension("spatial")

'CheckedOut'

generate masked singlepart BEC zones to create GPE with

In [22]:
BEC = "Terr_BEC_Subzones"
mask = "Terr_ParksConsComplexes"
#dissolveBEC = "dissolveBEC"

eraseOutput = "Terr_BEC_Erase"
mtsOutput = "Terr_BEC_Subzones_Multipart"

Testing subzone buffer

In [23]:
#erasedBEC = arcpy.Erase_analysis(BEC, mask, eraseOutput)
#arcpy.MultipartToSinglepart_management(erasedBEC, mtsOutput)

create two features classes: one for the non-complex protected areas, and one for the dissolved complexes

In [24]:
parkLayer = "Terr_ParksConsComplexes"

selection = 'Complex IS NOT NULL'

selected = arcpy.SelectLayerByAttribute_management(parkLayer, "NEW_SELECTION", selection)
complexes = arcpy.CopyFeatures_management(selected, os.path.join(scratch, "onlyComplexes.shp"))

dissolveComplexes = arcpy.Dissolve_management(complexes, "onlyComplexes",
                          "Complex", "", "MULTI_PART", 
                          "DISSOLVE_LINES")

arcpy.Delete_management(complexes)

selection2 = 'Complex IS NULL'
selected2 = arcpy.SelectLayerByAttribute_management(parkLayer, "NEW_SELECTION", selection2)
parks = arcpy.CopyFeatures_management(selected2, "onlyParks")
print("done")

done


In [25]:
regex = re.compile('[^a-zA-Z]')

with arcpy.da.UpdateCursor(parks, "PROTECTED_LANDS_NAME") as cursor:
    #removes all non-alphabetic characters for each polygon in the PROTECTED_LANDS_NAME field
    for row in cursor:
        row[0] = regex.sub('', row[0].title())

        # Update the cursor with the updated list
        cursor.updateRow(row)

generates the BEC GPE for a park, the select by attribute needs to be changed to updateCursor

In [26]:
def becGpeCreation(parkLayer, park, field, outList):
    #makes it simpler to do a select by attribute
    field = arcpy.AddFieldDelimiters(parkLayer, field)
    selection = "{field} = '{val}'".format(field=field, val=park)

    #removes all non alphabetic characters from the park name
    regex = re.compile('[^a-zA-Z]')
    gdbName = regex.sub('', park)

    #creates a geodatabase and saves a file name
    #if you fuck up again and delete everything, change outputs in this line to gdbLocation and uncomment it
    #in the initialization cell
    newGdb = os.path.join(outputs, gdbName)
    if not os.path.isdir(newGdb):
        os.makedirs(os.path.join(newGdb))
    parkSaveName = os.path.join(str(newGdb), gdbName + ".shp")

    outList.append(parkSaveName)

    #selected parameterized park, saves it to the new geodatabase
    selected = arcpy.SelectLayerByAttribute_management(parkLayer, "NEW_SELECTION", selection)
    parkFeature = arcpy.CopyFeatures_management(selected, parkSaveName)

    #generates the GPE, saves it to the new geodatabase
    selectBEC = arcpy.SelectLayerByLocation_management(mtsOutput, 'intersect', parkFeature)
    becSaveName = os.path.join(str(newGdb), gdbName + "_BEC.shp")
    arcpy.CopyFeatures_management(selectBEC, becSaveName)
    
    becList.append(becSaveName)

code to generate unique values in selected field: a workaround instead of using updateCursor for my specific use case

In [27]:
def bufferProp(inShape, saveLoc, UID, proportion = 1, sensitivity = .05, adjustRate = .05):
    #creates a buffer based on the percent of the area of the input shape, not including the original geometry
    #it also masks out the original shape and other shapes based on a mask parameter, and does not include what
    #is masked out in the proportions
    
    ###parameters
    #inShape: the input feature class or shapefile
    #saveLoc: name of the output feature class or shapefile
    #clip_by: valid locations for the buffer to go, may 
    #         need to be changed to a mask/erase to improve compute times 
    #UID: the unique identifier in the input table for each feature
    #proportion: how much of the area 
    
    #clip_by = dissolveBEC

    field_names = [f.name for f in arcpy.ListFields(inShape)]
    if "buffer_len" not in field_names:
        arcpy.AddField_management(inShape, "buffer_len", "DOUBLE")
        
    lowerBound = proportion - sensitivity
    upperBound = proportion + sensitivity
    
    fields = [UID, "SHAPE@AREA", "buffer_len"]
    
    gpeList = []
    
    with arcpy.da.UpdateCursor(inShape, fields) as cursor:
        for row in cursor:
            
            UID_ten = UID[:10]
            
            regex = re.compile('[^a-zA-Z]')
            fixedName = regex.sub('', row[0])
            
            clip_by = os.path.join(outputs, fixedName, fixedName + "_BEC.shp") 
            gpeSaveName = os.path.join(outputs, fixedName, fixedName + "_GPE.shp")
            
            iters = 0

            geomArea = row[1]
            r1 = math.sqrt(geomArea / math.pi)
            r2 = math.sqrt(geomArea * 2 / math.pi)
            row[2] = r2 - r1
            
            #potential to save an iteration by setting the upper to row[2] as defined here ^
            #since no parks will ever be an exact circle, and this radius based starting point
            #being based on areas of circles
            
            ##lower = 1
            ##upper = geomArea
            ##middle = row[2]
            
            ##upper = row[2]
            ##middle = (upper + lower) / 2

            ratio = None
            while (ratio is None or 
                   ratio < lowerBound or 
                   ratio > upperBound):
                
                iters += 1

                if ratio == None:
                    print("first attempt: no adjustment")
                elif ratio > proportion:
                    #print(ratio)
                    #print("decreasing buffer length by: " + str(sensitivity * 100) + "%")
                    row[2] *= (1 - adjustRate)
                    
                    ##upper = middle
                    ##middle = (lower + upper) / 2
                    ##row[2] = middle                 
                    
                elif ratio < proportion:
                    #print(ratio)
                    #print("increasing buffer length by: " + str(sensitivity * 100) + "%")
                    row[2] *= (1 + adjustRate)
                    
                    ##lower = middle
                    ##middle = (lower + upper) / 2
                    ##row[2] = middle

                cursor.updateRow(row)
                print(ratio)
                                        
                buffered = arcpy.Buffer_analysis(inShape, os.path.join(scratch, "interBuffer.shp"), "buffer_len")
                
                #this is where there is a timesave, for some reason clipping takes forever
                if ratio != None:
                    buffered = arcpy.Clip_analysis(buffered, clip_by, r"memory\BufferClip")
               
                with arcpy.da.SearchCursor(buffered, [UID_ten, "SHAPE@AREA"]) as SCur:
                    for SRow in SCur:
                        if SRow[0] == row[0]:
                            ratio = SRow[1] / geomArea
                            if lowerBound < ratio < upperBound:
                                print("final ratio: " + str(ratio))
                                print("number of iterations: " + str(iters))
                                
                                arcpy.CopyFeatures_management(buffered, gpeSaveName)
                                gpeList.append(gpeSaveName)
                                    
    arcpy.Delete_management(os.path.join(scratch, "interBuffer.shp"))
    #merged = arcpy.Merge_management(gpeList, saveLoc)

    return gpeList

In [28]:
def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return ({row[0] for row in cursor})

In [68]:
def subzones(ppaList, gpeList):
    expression = "Shape_Area < 1000"
    for ppa, gpe in zip(ppaList, gpeList):
        #generates save names for intersect
        folder = ppa.rsplit("\\", 1)[0]
        ppaName = ppa.rsplit('\\', 1)[-1].split(".")[0]
        gpeName = gpe.rsplit('\\', 1)[-1].split(".")[0]
        ppaSave = os.path.join(folder, ppaName + "_Subzones.shp")
        gpeSave = os.path.join(folder, gpeName + "_Subzones.shp")
        
        arcpy.CalculateField_management(ppa, "ppa_gpe", "'PPA'", "PYTHON3", field_type = "TEXT")
        arcpy.CalculateField_management(gpe, "ppa_gpe", "'GPE'", "PYTHON3", field_type = "TEXT")
        
        paceSave = os.path.join(folder, ppaName + "_PACE.shp")
        pace = arcpy.Merge_management([ppa, gpe], paceSave)

        intersectedPace = arcpy.Intersect_analysis([pace, "Terr_BEC_Subzones"], r"scratch\intersectedPace.shp")
        
        
        singlePart = arcpy.MultipartToSinglepart_management(r"scratch\intersectedPace.shp",
                                       r"scratch\paceSinglepart.shp")
        
        arcpy.CalculateGeometryAttributes_management(singlePart, 
                                                     [["Shape_Area", "AREA"]], 
                                                     area_unit = "SQUARE_METERS")
        
        selection = arcpy.SelectLayerByAttribute_management(singlePart, 
                                                "NEW_SELECTION", 
                                                expression)        
        paceSaveSub = os.path.join(folder, ppaName + "_PACE_Subzones.shp")
        arcpy.Eliminate_management(selection, paceSaveSub)
                
        arcpy.CalculateField_management(paceSaveSub, "myFID", "!FID!", "PYTHON3", field_type = "LONG")
        

In [30]:
#parkNames = unique_values(parks, "PROTECTED_LANDS_NAME")
#complexNames = unique_values(dissolveComplexes, "Complex")
parkNames = ("JoffreLakesPark", "MountSeymourPark", "StuartRiverParkLowerSite")
complexNames = ("Gar", "Spats", "Strath")
parkNames

('JoffreLakesPark', 'MountSeymourPark', 'StuartRiverParkLowerSite')

In [42]:
#initialize lists to merge at end
parkList = []
becList = []

#becGpeCreation(parks, "EkwanLakeProtectedArea", "PROTECTED_LANDS_NAME")

for park in parkNames:
    becGpeCreation(parks, park, "PROTECTED_LANDS_NAME", parkList)
    
    
    
print("done iteration, now merging")
    
#merge list of parks
testParks = arcpy.Merge_management(parkList, "parks")

print("done merging, running GPE buffer \n")

parkGPEs = bufferProp(testParks, "parkGPEs", "PROTECTED_")

print("\n done w/ proportional buffer clipping shenanigans")



done iteration, now merging
done merging, running GPE buffer 

first attempt: no adjustment
None
2.230917465224989
0.7956801848963748
0.8405497993715642
0.8883598801506545
0.9394145425099231
final ratio: 0.9938860412879031
number of iterations: 6
first attempt: no adjustment
None
2.6808755307520804
1.1462764687196794
1.1099979212606261
1.0745244314550613
final ratio: 1.0400234660185494
number of iterations: 5
first attempt: no adjustment
None
3.8188537850376063
2.5098401396090337
2.3855208563651447
2.267851456033138
2.1565136814045998
2.051063263501247
1.951157943936906
1.8564111139669128
1.7664702244458672
1.6809509468068633
1.5996240026982627
1.5224363371571803
1.4491023811314667
1.3794077766707034
1.3132335044147811
1.250360337570415
1.1905696277513462
1.1337982040154424
1.079723708209202
final ratio: 1.0282214196948305
number of iterations: 20

 done w/ proportional buffer clipping shenanigans


In [69]:
subzones(parkList, parkGPEs)

print("subzones generated")

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

subzones generated


In [37]:
#initialize lists to merge at end
complexesList = []
becList = []

for Complex in complexNames:
    becGpeCreation(dissolveComplexes, Complex, "Complex", complexesList)
    
print("done iteration, now merging")
    
#merge list of complexes
testComplexes = arcpy.Merge_management(complexesList, "complexes")

print("done merging, running GPE buffer \n")

complexesGPEs = bufferProp(testComplexes, "complexesGPEs", "Complex")

print("\n done w/ proportional buffer clipping shenanigans")



done iteration, now merging
done merging, running GPE buffer 

first attempt: no adjustment
None
2.7632034847992535
1.105535478564001
1.0812538231733015
1.056469811810365
final ratio: 1.0302690561907342
number of iterations: 5
first attempt: no adjustment
None
3.4745537613085524
1.405925682501722
1.3684220891115564
1.3305004450392726
1.2931194852283419
1.2545137372702544
1.2166477921721386
1.1800420072974396
1.1440746774322874
1.1093206272999518
1.0755474465658754
final ratio: 1.042339250334561
number of iterations: 12
first attempt: no adjustment
None
2.605802290786905
1.0713754827186148
final ratio: 1.03353851515969
number of iterations: 3

 done w/ proportional buffer clipping shenanigans


In [70]:
subzones(complexesList, complexesGPEs)

print("subzones generated")

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

subzones generated


myDict = {testParks: ("PPA", "Park"), 
          parkGPEs: ("GPE", "Park"),
          testComplexes: ("PPA", "Complex"),
          complexesGPEs: ("GPE", "Complex")}

shapeList = list(myDict.keys())

for key, value in myDict.items():   
    
    arcpy.AddField_management(key, "PPA_GPE", "TEXT")
    arcpy.AddField_management(key, "Park_Complex", "TEXT")
    
    with arcpy.da.UpdateCursor(key, ["PPA_GPE", "Park_Complex"]) as cursor:
        for row in cursor:
            row[0] = value[0]
            row[1] = value[1]
            cursor.updateRow(row)
        
PPA_GPEs = arcpy.Merge_management(shapeList, "PPA_GPEs")


parkFieldDict = {'ADMIN_AREA': 'ADMIN_AREA_SID', 
         'PROTECTED_': 'PROTECTED_LANDS_NAME', 
         'PROTECTED1': 'PROTECTED_LANDS_CODE', 
         'PROTECTE_1': 'PROTECTED_LANDS_DESIGNATION', 
         'ORCS_SECON': 'ORCS_SECONDARY', 
         'IUCN_Categ': 'IUCN_Category_Code', 
         'Park_Categ': 'Park_Category_Code', 
         'Establishe': 'Established_Date'}    

for key, value in parkFieldDict.items():
    arcpy.AlterField_management(PPA_GPEs, key, value, value)
    
    

arcpy.DeleteField_management(PPA_GPEs, ["Shape_Leng", "BUFF_DIST", "ORIG_FID"])     

# At this point, GPEs are generated. After this it is clipping the various variables to the PACEs. Theoretically, this could be a different notebook.


In [2]:
utm_7s = r"H:\utm\UTM_7S"
utm_8s = r"H:\utm\UTM_8S"
utm_9s = r"H:\utm\UTM_9S"
utm_10s = r"H:\utm\UTM_10S"
utm_11s = r"H:\utm\UTM_11S"

ntemsLocs = [utm_7s, utm_8s, utm_9s, utm_10s, utm_11s]
#print(ntemsLocs)

In [3]:
dist_7s = r"H:\utm\UTM_7S\Results\Change_attribution\Changes_attributed_logic_rules"
dist_8s = r"H:\utm\UTM_8s\Results\Change_attribution\Changes_attributed_logic_rules"
dist_9s = r"H:\utm\UTM_9s\Results\Change_attribution\Changes_attributed_logic_rules"
dist_10s = r"H:\utm\UTM_10s\Results\Change_attribution\Changes_attributed_logic_rules"
dist_11s = r"H:\utm\UTM_11s\Results\Change_attribution\Changes_attributed_logic_rules"

distLocs = [dist_7s, dist_8s, dist_9s, dist_10s, dist_11s]

In [4]:
for saveFolder, distFolder in zip(ntemsLocs, distLocs):
    
    shpList = []
    for file in os.listdir(distFolder):
        if file.endswith(".shp"):
            shpList.append(os.path.join(distFolder, file))
    
    distSaveLoc = os.path.join(saveFolder, "Change_attribution_logic_rules")
    if not os.path.isdir(distSaveLoc):
        os.makedirs(distSaveLoc)
        
    for shp in shpList:
        splitName = shp.rsplit('\\', 1)[-1].split(".")[0]
        saveName = os.path.join(distSaveLoc, splitName + ".tif")
        #print("running polygon to raster")
        if not os.path.isfile(saveName):
            field_names = [f.name for f in arcpy.ListFields(shp)]

            if "r_class" not in field_names:
                arcpy.AddField_management(shp, "r_class", "SHORT")

            with arcpy.da.UpdateCursor(shp, ["CLASS_LOGI", "r_class"]) as cursor:
                for row in cursor:
                    if row[0] == "Unclassified":
                        row[1] = 1000
                    if row[0] == "Fire":
                        row[1] = 1001
                    if row[0] == "Harvesting":
                        row[1] = 1002
                    if row[0] == "Lcondition":
                        row[1] = 1003
                    if row[0] == "Road":
                        row[1] = 1004
                    if row[0] == "Ag":
                        row[1] = 1005

                    cursor.updateRow(row)
        
        
            arcpy.PolygonToRaster_conversion(shp, "r_class", saveName, cellsize = 30)
        print("done " + saveName)
        
        
    print("done " + saveFolder)

done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1985.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1986.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1987.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1988.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1989.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1990.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1991.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1992.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1993.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1994.tif
done H:\utm\UTM_7S\Change_attr

In [5]:
#ppaList = parkList + complexesList
#ppaList

In [6]:
structVars = ["basal_area", "elev_cv", "gross_stem_volume",
             "loreys_height", "percentage_first_returns_above_2m", 
             "total_biomass"]

#unusedStructVars = ["elev_mean", "elev_p95", "elev_stddev", "percentage_first_returns_above_mean"]

input parameters: years, variables, ppas

In [9]:
#inVars = ["HMM", "basal_area"]
inVars = ["HMM", "disturbance", "disturbance_all", "fragstats", "elevation", "nightlights"] + structVars
#years = list(range(1984, 2020))
years = [2015]
ppaList = ['C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\Gar\\Gar_PACE_Subzones.shp',
           'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\Spats\\Spats_PACE_Subzones.shp',
           'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\StuartRiverParkLowerSite\\StuartRiverParkLowerSite_PACE_Subzones.shp',
           'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\JoffreLakesPark\\JoffreLakesPark_PACE_Subzones.shp',
           'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\MountSeymourPark\\MountSeymourPark_PACE_Subzones.shp',
           'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\Strath\\Strath_PACE_Subzones.shp'
        ]

inVars = ["nightlights"]
#ppaList = ['C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\Gar\\Gar_PACE_Subzones.shp']
#years = list(range(1985, 2019))
print(inVars)
print(years)
print(ppaList)

['nightlights']
[2015]
['C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\Gar\\Gar_PACE_Subzones.shp', 'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\Spats\\Spats_PACE_Subzones.shp', 'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\StuartRiverParkLowerSite\\StuartRiverParkLowerSite_PACE_Subzones.shp', 'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\JoffreLakesPark\\JoffreLakesPark_PACE_Subzones.shp', 'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\MountSeymourPark\\MountSeymourPark_PACE_Subzones.shp', 'C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\Strath\\Strath_PACE_Subzones.shp']


function to clip the ntems lcc and structure data, all inputs must be in list format

In [12]:
def ntemsClipping(ppaList, inVars, years):
    
    numZones = 5
    
    x = 1
    iters = len(years) * len(inVars) * len(ppaList)
    
    
    df = pd.DataFrame(columns = ("park", "var", "year", "exists"))
    
    for ppa in ppaList:
        ppaName = ppa.rsplit('\\', -1)[-1].split(".")[0].rsplit("_", 1)[0]

        for Var in inVars:
            
            VarSave = Var
                
            parkFolder = os.path.normpath(ppa + os.sep + os.pardir)
            saveFolder = os.path.join(parkFolder, "rasters")
            
            if not os.path.isdir(saveFolder):
                os.makedirs(os.path.join(saveFolder))

            for year in years:
                print(str(x), "/", str(iters) + " iterations:", ppaName, Var, year, "working", end = "\r")

                intSaveLocs = []
                
                finalSaveName = ppaName + "-" + str(year) + "-" + VarSave + ".tif"
                if Var == "elevation":
                    finalSaveName = ppaName + "-elevation.tif"
                if Var == "disturbance_all":
                    finalSaveName = ppaName + "-disturbance_all.tif"
                if not os.path.isfile(os.path.join(saveFolder, finalSaveName)):
                    
                    #for variables that have a single combined layer across bc_albers
                    if Var == "fragstats":
                        loc = r"H:\Muise\F_noF_other"
                        varLoc = os.path.join(loc, "F-noF_BC_" + str(year) + ".tif")
                        final = arcpy.sa.ExtractByMask(varLoc, ppa)
                        final.save(os.path.join(saveFolder, finalSaveName))
                        
                    elif Var == "elevation":
                        #elevation
                        loc = r"H:\Muise\Elevation"
                        varLoc = os.path.join(loc, "DEM_BC.tif")
                        paceElev = arcpy.sa.ExtractByMask(varLoc, ppa)
                        paceElev.save(os.path.join(saveFolder, finalSaveName))
                        
                        #slope
                        finalSaveName = ppaName + "-slope.tif"
                        
                        paceSlope = Slope(paceElev)
                        paceSlope.save(os.path.join(saveFolder, finalSaveName))
                        
                        #aspect
                        finalSaveName = ppaName + "-aspect.tif"
                        
                        paceAspect = Aspect(paceElev)
                        paceAspect.save(os.path.join(saveFolder, finalSaveName))
                        
                    elif Var == "nightlights":
                        loc = r"H:\Muise\nightlights"
                        varLoc = os.path.join(loc, "bc-albers_nightlights_" + str(year) + "_reclass.tif")
                        
                        final = arcpy.sa.ExtractByMask(varLoc, ppa)
                        final.save(os.path.join(saveFolder, finalSaveName))
                    
                                           
                    #for variables that do not have a single combined layer across bc_albers  
                    else:
                        for i in range(numZones):

                            zoneNum = str(7 + i) + "S_"
                            zone = "UTM_" + str(7 + i) + "S"
                            zoneDistAll = "UTM" + str(7 + i) + "S"

                            #this has to be changed to work on other variables

                            if Var is "HMM":
                                loc = r"H:\VLCE"
                                varLoc = os.path.join(loc, zone, Var, "LC_Class_HMM_" + zoneNum + str(year) + ".dat")

                            elif Var in structVars:
                                loc = r"H:\Structure"
                                varLoc = os.path.join(loc, zone, Var, "UTM_" + zoneNum + Var + "_" + str(year) + ".dat")

                            elif Var is "disturbance":
                                loc = r"H:\utm"
                                varLoc = os.path.join(loc, zone, "Change_attribution_logic_rules", "Logic_Rules_Change_Attribution_UTM_" + zoneNum + str(year) + ".tif")
                            
                            elif Var is "disturbance_all":
                                loc = r"H:\utm"
                                varLoc = os.path.join(loc, zone, "Results", "Change_attribution", "Attribution_"  + zoneDistAll + "_v2.dat")


                            if not os.path.isfile(os.path.join(saveFolder, finalSaveName)): 
                                #print(varLoc)
                                try:
                                    interim = arcpy.sa.ExtractByMask(varLoc, ppa)
                                    intSaveLoc = os.path.join(scratch, "test_" + zone + ".tif")
                                    interim.save(intSaveLoc)
                                    intSaveLocs.append(intSaveLoc)

                                except arcpy.ExecuteError:
                                    continue



                        
                        #print(intSaveLocs)
                        try:
                            if Var in structVars:
                                arcpy.CreateRasterDataset_management(scratch, finalSaveName, "30",
                                     "16_BIT_UNSIGNED", number_of_bands = 1)

                                arcpy.Mosaic_management(intSaveLocs, os.path.join(scratch, finalSaveName))
                                final = arcpy.sa.ExtractByMask(os.path.join(scratch, finalSaveName), ppa)
                                forMaskLoc = r"H:\Muise\forestMask"
                                forMaskFile = "ForMask_UTM-7s-11S_" + str(year) + ".tif"
                                yearForMask = os.path.join(forMaskLoc, forMaskFile)
                                final = Times(final, yearForMask)
                                final.save(os.path.join(saveFolder, finalSaveName))

                            elif Var is "disturbance":
                                arcpy.CreateRasterDataset_management(scratch, finalSaveName, "30",
                                         "16_BIT_UNSIGNED", number_of_bands = 1)

                                interimMosaic = arcpy.Mosaic_management(intSaveLocs, os.path.join(scratch, finalSaveName))
                                interimMosaic = Con(IsNull(interimMosaic), 0, interimMosaic)
                                final = arcpy.sa.ExtractByMask(interimMosaic, ppa)

                                final.save(os.path.join(saveFolder, finalSaveName))

                            elif Var is "HMM":
                                arcpy.CreateRasterDataset_management(scratch, finalSaveName, "30",
                                         "16_BIT_UNSIGNED", number_of_bands = 1)

                                arcpy.Mosaic_management(intSaveLocs, os.path.join(scratch, finalSaveName))
                                final = arcpy.sa.ExtractByMask(os.path.join(scratch, finalSaveName), ppa)
                                final.save(os.path.join(saveFolder, finalSaveName))

                            else:
                                arcpy.CreateRasterDataset_management(scratch, finalSaveName, "30",
                                         "16_BIT_UNSIGNED", number_of_bands = 1)

                                arcpy.Mosaic_management(intSaveLocs, os.path.join(scratch, finalSaveName))
                                final = arcpy.sa.ExtractByMask(os.path.join(scratch, finalSaveName), ppa)
                                final.save(os.path.join(saveFolder, finalSaveName))

                            df.loc[x] = (ppaName, VarSave, year, "worked")
                        except:
                            df.loc[x] = (ppaName, VarSave, year, "did not work")
                            continue
                else:
                    df.loc[x] = (ppaName, VarSave, year, "already exists")
                x += 1
                
    return df

In [15]:
outDf = ntemsClipping(ppaList, inVars, years)

In [16]:
pd.set_option("display.max_rows", None)
display(outDf)

,park,var,year,exists
1,Gar_PACE,nightlights,2015,already exists
2,Spats_PACE,nightlights,2015,already exists
3,StuartRiverParkLowerSite_PACE,nightlights,2015,already exists
4,JoffreLakesPark_PACE,nightlights,2015,already exists
5,MountSeymourPark_PACE,nightlights,2015,already exists
6,Strath_PACE,nightlights,2015,already exists


# Variables are now clipped to the PACE, next need to aggregate average data to subzones: New notebook start point?

In [2]:
structVars = ["basal_area", "elev_cv", "gross_stem_volume",
             "loreys_height", "percentage_first_returns_above_2m", 
             "total_biomass"]
meanVars = structVars + ["elevation", "slope", "aspect", "insolation"]

meanVars = tuple([s + ".tif" for s in meanVars])

In [19]:
def zStatsMean(fName, subzone):
    metadata = fName.split("\\")[-1].split(".")[0].split("-")
    var = metadata[-1]
    if var in structVars:
        year = metadata[1]
    else:
        year = None
    tSaveLoc = os.path.join(scratch, fName.split("\\")[-1].split(".")[0] + ".dbf")
    zonalOut = ZonalStatisticsAsTable(subzone, "myFID", fName, tSaveLoc, "DATA", "MEAN")
    arcpy.JoinField_management(zonalOut, "myFID", subzone, "myFID", ["ppa_gpe", "ZONE", "SUBZONE", "VARIANT", "PHASE", "NATURAL_DI", "Shape_Area"])
    if var in structVars:
        arcpy.CalculateField_management(zonalOut, "year", year, field_type = "SHORT")
    arcpy.CalculateField_management(zonalOut, "var", "var", field_type = "TEXT")
    arcpy.CalculateField_management(zonalOut, "park", "parkName", field_type = "TEXT")
    return zonalOut

In [22]:
d = 'outputs'
parkFolders = [os.path.join(wd, d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]

outDbfs = []

for park in parkFolders:
    parkName = park.rsplit("\\", 1)[-1]
    #print(parkName)
    subzone = os.path.join(park, parkName + "_PACE_Subzones.shp")
    #print(subzone)
    rasterLoc = os.path.join(park, "rasters")
    #print(rasterLoc)
    for file in os.listdir(rasterLoc):
        #for meanVar in meanVars:
        if file.endswith(meanVars):
            print("working on", file, end = '\x1b[2K\r')
            fileLoc = os.path.join(rasterLoc, file)
            metadata = file.split(".")[0].split("-")
            var = metadata[-1]
            outDbfs.append(zStatsMean(fileLoc, subzone))

            
            


        #if file.endswith(propVars):
            #propLocs.append(os.path.join(rasterLoc, file))
mergedDbf = arcpy.Merge_management(outDbfs, "scratch/structure.dbf")            
arcpy.TableToTable_conversion(mergedDbf, "outputCsvs", "meanVars.csv")

for file in os.listdir(scratch):
    os.remove(os.path.join(scratch, file))
print("done!")

done!ng on StuartRiverParkLowerSite_PACE-slope.tifs.tifrns_above_2m.tif
